In [ ]:
import os
import gc
import logging
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve, accuracy_score, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn import svm
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.base import clone
from sklearn.pipeline import Pipeline

from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateSchedulers
from keras import callbacks
from keras import metrics
from keras.optimizers import Adam
from keras import backend as K
import keras
from keras.models import Model, Sequential
from keras.models import model_from_json
from keras import regularizers
from keras.losses import binary_crossentropy
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf

In [ ]:
%matplotlib inline

In [ ]:
start_time = time.time()

In [ ]:
train_df = pd.read_csv('../input/train.csv')

In [ ]:
test_df = pd.read_csv('../input/test.csv')

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
from sklearn.preprocessing import power_transform
features = [c for c in train.columns if c not in ['ID_code', 'target']]
for feature in features:
    train_features['mean_'+feature] = (train_features[feature].mean()-train_features[feature])
    train_features['z_'+feature] = (train_features[feature] - train_features[feature].mean())/train_features[feature].std(ddof=0)
    train_features['sq_'+feature] = (train_features[feature])**2
    train_features['sqrt_'+feature] = (train_features['sq_'+feature])**(1/4)
    train_features['log_'+feature] = np.log(train_features['sq_'+feature]+10)/2

In [ ]:
train_features.head()

In [ ]:
sc = StandardScaler()
train_features = sc.fit_transform(train_features)

In [ ]:
gc.collect()

In [ ]:
input_dim = train_features.shape[1]
input_dim

In [ ]:
class printAUC(callbacks.Callback):
    def __init__(self, X_train, y_train):
        super(printAUC, self).__init__()
        self.bestAUC = 0
        self.X_train = X_train
        self.y_train = y_train
        
    def on_epoch_end(self, epoch, logs={}):
        pred = self.model.predict(np.array(self.X_train))
        auc = roc_auc_score(self.y_train, pred)
        print("Train AUC: " + str(auc))
        #pred = self.model.predict(self.validation_data[0])
        #auc = roc_auc_score(self.validation_data[1], pred)
        #print ("Validation AUC: " + str(auc))
        if (self.bestAUC < auc) :
            self.bestAUC = auc
            self.model.save("bestNet.h5", overwrite=True)
        return

In [ ]:
def step_decay(epoch):
    initial_rate
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
       self.losses = []
       self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
       self.losses.append(logs.get('loss'))
       self.lr.append(step_decay(len(self.losses)))

In [ ]:
min_max_scaler = MinMaxScaler()

In [ ]:
standart_scaler = StandardScaler()

In [ ]:
robust_scaler = RobustScaler()

In [ ]:
#scaled_train_values = min_max_scaler.fit_transform(train_df[train_df.columns.drop(['ID_code', 'target'])])
scaled_train_values = standart_scaler.fit_transform(train_df[train_df.columns.drop(['ID_code', 'target'])])
#scaled_train_values = robust_scaler.fit_transform(train_df[train_df.columns.drop(['ID_code', 'target'])])

In [ ]:
target_values = train_df['target'].values

In [ ]:
train_values, holdout_test_values, train_target_values, holdout_test_target_values = train_test_split(
    #scaled_train_values,
    train_df[train_df.columns.drop(['ID_code', 'target'])].values,
    target_values,
    test_size=0.2,
    random_state=0
)

In [ ]:
print(train_values.shape)
print(train_target_values.shape)
print(holdout_test_values.shape)
print(holdout_test_target_values.shape)

In [ ]:
#test_df.head()

In [ ]:
ID_code = test_df['ID_code'].values

In [ ]:
print(ID_code[:10])

In [ ]:
#tf.py_func

In [ ]:
batch_size = 100

In [ ]:
def auc(y_true, y_pred):
#def auc(y_pred, y_true):
    #print(y_true[:5])
    #print(y_pred[:5])  
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)
    #return tf.py_func(roc_auc_score, (y_true, y_pred), tf.int8)

Провести mean_shift кластеризацию на множестве переменных (features)?

In [ ]:
#submission_predicts = sequential_nn_model.predict(test_df[test_df.columns.drop('ID_code')].values)

In [ ]:
#submission_predicts.shape

In [ ]:
#print(submission_predicts[:5])
#submission_predicts_values = submission_predicts[:, 0]

In [ ]:
#submission_predicts_values.shape

In [ ]:
#np.all(submission_predicts_values == 0)

In [ ]:
#submission_df = pd.DataFrame({'ID_code': ID_code, 'target': submission_predicts_values.astype('float32')})

In [ ]:
#np.any(submission_predicts_values == 1)

In [ ]:
#np.all(submission_predicts_values > 0)

In [ ]:
#np.all(submission_predicts_values < 0)

In [ ]:
#np.any(submission_predicts_values > 0)

In [ ]:
#np.any(submission_predicts_values < 0)

In [ ]:
#submission_predicts_values[submission_predicts_values > 0].shape

In [ ]:
#submission_predicts_values.max()

In [ ]:
#submission_predicts_values.min()

In [ ]:
#submission_predicts_values[submission_predicts_values < 0.245].shape

In [ ]:
#submission_predicts_values[submission_predicts_values >= 0.245].shape

In [ ]:
#submission_predicts_values[submission_predicts_values < 0.245] = 0
#submission_predicts_values[submission_predicts_values >= 0.245] = 1

In [ ]:
#submission_predicts_values[submission_predicts_values == 1].shape

In [ ]:
#submission_df = pd.DataFrame({'ID_code': ID_code, 'target': submission_predicts_values.astype('float32')})

In [ ]:
#submission_df.to_csv('submission_mlp_1.csv', index=False)

In [ ]:
def detect_threshold(classes_ratio, step_size, predicted_values):
    threshold = predicted_values.min()
    values_below_threshold = (predicted_values[predicted_values < threshold]).shape[0]
    values_above_threshold = (predicted_values[predicted_values >= threshold]).shape[0]
    #threshold_classes_ratio =  values_above_threshold / values_below_threshold
    threshold_classes_ratio = 1
    
    while(threshold_classes_ratio > classes_ratio):
        threshold += step_size        
        values_below_threshold = (predicted_values[predicted_values < threshold]).shape[0]
        values_above_threshold = (predicted_values[predicted_values >= threshold]).shape[0]
        threshold_classes_ratio =  values_above_threshold / values_below_threshold
    predicted_values[predicted_values < threshold] = 0
    predicted_values[predicted_values >= threshold] = 1
    return threshold, predicted_values

In [ ]:
sequential_nn_model = None
#del sequential_nn_model
if sequential_nn_model:
    del sequential_nn_model
sequential_nn_model = Sequential()
sequential_nn_model.add(Dense(batch_size, input_dim=200, kernel_initializer='normal', activation='relu'))
sequential_nn_model.add(Dropout(0.4))
sequential_nn_model.add(Dense(batch_size, input_dim=100, kernel_initializer='normal', activation='sigmoid'))
sequential_nn_model.add(Dropout(0.4))
sequential_nn_model.add(Dense(batch_size, input_dim=50, kernel_initializer='normal', activation='relu'))
sequential_nn_model.add(Dropout(0.4))
sequential_nn_model.add(Dense(batch_size, input_dim=50, kernel_initializer='normal', activation='sigmoid'))
#sequential_nn_model.add(Dense(batch_size, input_shape=(100, 200), kernel_initializer='normal', activation='sigmoid'))
#sequential_nn_model.add(Dropout(0.76))
#sequential_nn_model.add(Dropout(0.24))
sequential_nn_model.add(Dropout(0.4))
sequential_nn_model.add(Dense(batch_size, input_dim=10, kernel_initializer='normal', activation='relu'))
sequential_nn_model.add(Dropout(0.4))
sequential_nn_model.add(Dense(batch_size, input_dim=10, kernel_initializer='normal', activation='sigmoid'))
sequential_nn_model.add(Dropout(0.4))
sequential_nn_model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

In [ ]:
#sequential_nn_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy', auc])
sequential_nn_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy', auc])

In [ ]:
start_time = time.time()
sequential_nn_model.fit(train_values, train_target_values, batch_size=100, epochs=40, verbose=1, validation_split=0.2)
print("Run time {} min".format((time.time() - start_time) / 60))

In [ ]:
loss_and_metrics = sequential_nn_model.evaluate(holdout_test_values, holdout_test_target_values, batch_size=100)

In [ ]:
loss_and_metrics
# [0.25161404045298696, 0.9094000032544136, 0.8320361980522218] [0.2493363002128899, 0.9096750013530255, 0.8369347876192563]
# [0.24582509476691483, 0.9101250021159649, 0.8396344886385845] [0.24802235754206778, 0.9094000029563903, 0.8385308386618173]
# [0.24494799628853797, 0.9102000007033348, 0.839084512352511]

In [ ]:
model_json = sequential_nn_model.to_json()
with open("sequential_nn_model_relu_droput024_lr001_sigmoid_40_epochs_2019-03-30-1.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
sequential_nn_model.save("sequential_nn_model_relu_dropout024_lr001_sigmoid_40_epochs_2019-03-30-1.h5")

In [ ]:
sequential_nn_model.save_weights("sequential_nn_model_weights_relu_dropout024_lr001_sigmoid_40_epochs_2019-03-30-1.h5")

In [ ]:
submission_predict_values = sequential_nn_model.predict(test_df[test_df.columns.drop('ID_code')].values)[:, 0]

In [ ]:
classes_ratio = train_df[train_df['target'] == 1].shape[0] / train_df[train_df['target'] == 0].shape[0]

In [ ]:
classes_ratio

In [ ]:
threshold, submission_predicts = detect_threshold(classes_ratio, 0.01, submission_predict_values)

In [ ]:
threshold

In [ ]:
submission_predicts[submission_predicts == 0].shape

In [ ]:
submission_predicts[submission_predicts == 1].shape

In [ ]:
submission_df = pd.DataFrame({'ID_code': ID_code, 'target': submission_predicts.astype('float32')})

In [ ]:
submission_df.to_csv('submission_mlp_3.csv', index=False)